# Progetto di Foundations of Computer Science - 2019

####  Importo le librerie necessarie

In [1]:
import pandas as pd
import numpy
import re
pd.set_option('display.max_columns', None)

### Leggo i file CSV dal PC di Matteo

In [2]:
ll = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loans_lenders.csv")

In [3]:
loans = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loans.csv")

In [4]:
lenders = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/lenders.csv")

In [5]:
isocodes = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/isocodes.csv")
isocodes = isocodes[['name','alpha-2']]
isocodes.columns = ['country_name','country_code']
df2 = pd.DataFrame([['Netherland Antilles','AN'], ['Serbia and Montenegro', 'CS'], ['Gaza', 'GZ'], ['Kosovo', 'XK']], columns=['country_name','country_code'])
isocodes = isocodes.append(df2, ignore_index=True)

***

###  1. Normalize the *loan_lenders* table. In the normalized table, each row must have one *loan_id* and one *lender*.

In [6]:
df = (ll.set_index(['loan_id']).apply(lambda x: x.str.split(',').explode()).reset_index())
df

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


***

###  2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing

Da sistemare, problema con date negative

In [6]:
loans['duration'] = (pd.to_datetime(loans.planned_expiration_time)-pd.to_datetime(loans.disburse_time)).dt.days

***


### 3. Find the lenders that have funded at least twice.

In [8]:
fund = df.groupby(['lenders']).size().reset_index()
fund.columns = ['lenders', 'num_of_fund']

In [9]:
fund[fund['num_of_fund'] >= 2]

,lenders,num_of_fund
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1639018,zyrorl,3
1639020,zzaman,11
1639021,zzanita,2
1639024,zzmcfate,56


***

### 4. For each country, compute how many loans have involved that country as borrowers.

In [10]:
loan_country = loans[loans['status'] == 'funded'][['country_name']]

In [107]:
country_nloan = loan_country.groupby('country_name').size().reset_index()
country_nloan.columns = ['country_name','num_of_loans']
country_nloan

,country_name,num_of_loans
0,Afghanistan,2264
1,Albania,2730
2,Armenia,11251
3,Azerbaijan,9271
4,Belize,213
...,...,...
89,Vanuatu,4
90,Vietnam,20813
91,Yemen,4017
92,Zambia,1195


***

### 5. For each country, compute the overall amount of money borrowed.

In [12]:
loan_country = loans[loans['status'] == 'funded'][['country_name','loan_amount']]
country_amount = loan_country.groupby('country_name').sum().reset_index()
country_amount

,country_name,loan_amount
0,Afghanistan,1899250.0
1,Albania,3703925.0
2,Armenia,17788000.0
3,Azerbaijan,13150000.0
4,Belize,148350.0
...,...,...
89,Vanuatu,9250.0
90,Vietnam,22219750.0
91,Yemen,3180725.0
92,Zambia,1763425.0


***

### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [13]:
overall = country_amount['loan_amount'].sum()
country_amount['percentage'] = country_amount['loan_amount']/overall*100
country_amount

,country_name,loan_amount,percentage
0,Afghanistan,1899250.0,0.175778
1,Albania,3703925.0,0.342803
2,Armenia,17788000.0,1.646305
3,Azerbaijan,13150000.0,1.217051
4,Belize,148350.0,0.013730
...,...,...,...
89,Vanuatu,9250.0,0.000856
90,Vietnam,22219750.0,2.056469
91,Yemen,3180725.0,0.294381
92,Zambia,1763425.0,0.163207


***

### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [14]:
loan_country = loans[loans['status'] == 'funded'][['country_name','loan_amount','disburse_time']]
loan_country['disburse_time'] = pd.to_datetime(loan_country['disburse_time']).dt.year

In [15]:
loan_country.groupby(['country_name', 'disburse_time']).size()

country_name  disburse_time
Afghanistan   2007.0            408
              2008.0            369
              2009.0            658
              2010.0            630
              2011.0            197
                               ... 
Zimbabwe      2013.0            422
              2014.0           2048
              2015.0            598
              2016.0            805
              2017.0           1047
Length: 745, dtype: int64

In [16]:
country_amount = loan_country.groupby(['country_name', 'disburse_time']).sum()
country_amount

loan_amount
country_name disburse_time             
Afghanistan  2007.0            194975.0
             2008.0            364300.0
             2009.0            568850.0
             2010.0            561850.0
             2011.0            195275.0
...                                 ...
Zimbabwe     2013.0            670625.0
             2014.0           1292775.0
             2015.0            720500.0
             2016.0            786200.0
             2017.0           1197050.0

[745 rows x 1 columns]

In [27]:
overall = country_amount['loan_amount'].sum()
country_amount['percentage'] = country_amount['loan_amount']/overall*100
country_amount

loan_amount  percentage
country_name disburse_time                         
Afghanistan  2007.0            194975.0    0.018048
             2008.0            364300.0    0.033722
             2009.0            568850.0    0.052656
             2010.0            561850.0    0.052008
             2011.0            195275.0    0.018076
...                                 ...         ...
Zimbabwe     2013.0            670625.0    0.062077
             2014.0           1292775.0    0.119666
             2015.0            720500.0    0.066693
             2016.0            786200.0    0.072775
             2017.0           1197050.0    0.110805

[745 rows x 2 columns]

***

### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [5]:
loan_lent = pd.merge(df, loans[loans['status'] == 'funded'][['loan_id','loan_amount']], how='left', on='loan_id')
loan_lent

,loan_id,lenders,loan_amount
0,483693,muc888,1225.0
1,483693,sam4326,1225.0
2,483693,camaran3922,1225.0
3,483693,lachheb1865,1225.0
4,483693,rebecca3499,1225.0
...,...,...,...
28293926,1206425,trogdorfamily7622,325.0
28293927,1206425,danny6470,325.0
28293928,1206425,don6118,325.0
28293929,1206486,alan5175,150.0


In [6]:
loan_lent = pd.merge(loan_lent, loan_lent.groupby('loan_id')['lenders'].count(), how='left', on='loan_id')
loan_lent.columns = ['loan_id', 'lenders', 'loan_amount', 'count']

In [7]:
loan_lent['ind_share'] = loan_lent['loan_amount']/loan_lent['count']

In [8]:
loan_lent.groupby('lenders')['ind_share'].sum().reset_index()

,lenders,ind_share
0,000,1672.618411
1,00000,1340.599304
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1639021,zzanita,87.500000
1639022,zzcyna7269,55.357143
1639023,zzinnia,38.000000
1639024,zzmcfate,2041.677920


***

### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [7]:
countries = lenders['country_code'].dropna().reset_index()
countries = countries.groupby('country_code').size().reset_index()
countries.columns = ['country_code', 'num']
countries = pd.merge(countries, isocodes, how='left', on='country_code')

In [8]:
countries['percentage'] = countries['num']/countries['num'].sum()

In [9]:
countries

,country_code,num,country_name,percentage
0,AD,15,Andorra,0.000017
1,AE,1043,United Arab Emirates,0.001171
2,AF,228,Afghanistan,0.000256
3,AG,8,Antigua and Barbuda,0.000009
4,AI,4,Anguilla,0.000004
...,...,...,...,...
229,YE,195,Yemen,0.000219
230,YT,2,Mayotte,0.000002
231,ZA,1051,South Africa,0.001180
232,ZM,65,Zambia,0.000073


In [10]:
df_temp = pd.merge(loans[['loan_id','loan_amount']], df, how='left', on='loan_id')
df_temp.columns = ['loan_id','loan_amount','permanent_name']
df_temp = pd.merge(df_temp,lenders[['permanent_name','country_code']], how='left', on='permanent_name')
df_temp

,loan_id,loan_amount,permanent_name,country_code
0,657307,125.0,spencer5657,US
1,657307,125.0,matthew8640,NaN
2,657307,125.0,larry71496105,NaN
3,657259,400.0,ltr,CA
4,657259,400.0,andrew5306,NaN
...,...,...,...,...
28326086,989240,175.0,jensdamsgaardvanar,NO
28326087,989240,175.0,david47598776,NaN
28326088,989240,175.0,bernd9221,NaN
28326089,989240,175.0,valeria98599473,NaN


In [11]:
df_temp = pd.merge(df_temp, df_temp.groupby('loan_id').size().reset_index(), how='left', on='loan_id')
df_temp.columns = ['loan_id','loan_amount','lenders','country_code','count']
df_temp['loan_amount'] = df_temp['loan_amount']/df_temp['count']
df_temp

,loan_id,loan_amount,lenders,country_code,count
0,657307,41.666667,spencer5657,US,3
1,657307,41.666667,matthew8640,NaN,3
2,657307,41.666667,larry71496105,NaN,3
3,657259,57.142857,ltr,CA,7
4,657259,57.142857,andrew5306,NaN,7
...,...,...,...,...,...
28326086,989240,35.000000,jensdamsgaardvanar,NO,5
28326087,989240,35.000000,david47598776,NaN,5
28326088,989240,35.000000,bernd9221,NaN,5
28326089,989240,35.000000,valeria98599473,NaN,5


In [12]:
df_temp2 = df_temp.dropna(subset=['country_code']).drop(columns=['count']) #NON HA NESSUN NAN, HO CONTROLLATO!
df_temp2

,loan_id,loan_amount,lenders,country_code
0,657307,41.666667,spencer5657,US
3,657259,57.142857,ltr,CA
24,659347,36.764706,jasonsamfield,US
41,656933,30.357143,john86857365,US
55,659605,25.000000,stephani8455,US
...,...,...,...,...
28325958,986789,31.250000,timothy5629,GB
28325974,987606,176.470588,sara7949,GB
28325991,988180,30.769231,joyce5432,US
28326015,989109,35.144928,ulrike5921,US


In [13]:
#Questi qua sono i prestiti per paese CONOSCIUTI!
df_temp2.groupby('country_code').sum().reset_index().drop(columns=['loan_id'])

,country_code,loan_amount
0,AD,115.151515
1,AE,116011.090445
2,AF,6260.801566
3,AL,732.294449
4,AM,1042.007307
...,...,...
190,WF,200.376984
191,XK,236.847527
192,YE,4851.012333
193,ZA,24175.056019


In [14]:
#Totale dei soldi prestati dai NaN
totale = df_temp[df_temp['country_code'].isnull()]['loan_amount'].sum()
totale

1120959740.8815444

In [15]:
countries['money_lent'] = countries['percentage']*totale
countries #DA SOMMARE I SOLDI DI QUELLI CONOSCIUTI

,country_code,num,country_name,percentage,money_lent
0,AD,15,Andorra,0.000017,1.888115e+04
1,AE,1043,United Arab Emirates,0.001171,1.312869e+06
2,AF,228,Afghanistan,0.000256,2.869934e+05
3,AG,8,Antigua and Barbuda,0.000009,1.006994e+04
4,AI,4,Anguilla,0.000004,5.034972e+03
...,...,...,...,...,...
229,YE,195,Yemen,0.000219,2.454549e+05
230,YT,2,Mayotte,0.000002,2.517486e+03
231,ZA,1051,South Africa,0.001180,1.322939e+06
232,ZM,65,Zambia,0.000073,8.181830e+04


In [16]:
countries = pd.merge(countries, df_temp2.groupby('country_code').sum().reset_index().drop(columns=['loan_id']), how='left', on='country_code')

In [24]:
countries = countries.fillna(0)
countries['tot_money_lent'] = countries['money_lent'] + countries['loan_amount']
countries.sort_values("tot_money_lent", axis = 0, ascending = True, inplace = True, na_position ='last') 
countries

,country_code,num,country_name,percentage,money_lent,loan_amount,tot_money_lent
199,TF,1,French Southern Territories,0.000001,1.258743e+03,0.000000e+00,1.258743e+03
86,GW,1,Guinea-Bissau,0.000001,1.258743e+03,0.000000e+00,1.258743e+03
157,NU,1,Niue,0.000001,1.258743e+03,0.000000e+00,1.258743e+03
98,IO,1,British Indian Ocean Territory,0.000001,1.258743e+03,0.000000e+00,1.258743e+03
141,MS,1,Montserrat,0.000001,1.258743e+03,0.000000e+00,1.258743e+03
...,...,...,...,...,...,...,...
52,DE,16007,Germany,0.017975,2.014870e+07,1.940454e+06,2.208915e+07
13,AU,37103,Australia,0.041664,4.670314e+07,3.087948e+06,4.979109e+07
72,GB,38380,United Kingdom of Great Britain and Northern I...,0.043097,4.831056e+07,2.506089e+06,5.081665e+07
35,CA,67970,Canada,0.076325,8.555676e+07,4.530814e+06,9.008758e+07


***
***
***

# Robe vecchie (probabilmente sbagliate)

### 4. For each country, compute how many loans have involved that country as borrowers

Rinomino le colonne di country_coords. Inoltre arrotondo le coordinate in loan_coords e le rendo intere in modo da poter effettuare il merge.

In [6]:
country_coords = GEconV4[['COUNTRY','LAT','LONGITUDE']]
country_coords.columns = ['country','latitude','longitude']

In [7]:
loan_coords[['latitude','longitude']]=loan_coords[['latitude','longitude']].apply(round).astype('int64')

Effettuo il merge in modo da ottenere un dataframe che contenga i loan_id per i rispettivi paesi

In [8]:
loan_country = pd.merge(loan_coords, country_coords.drop_duplicates(subset=['latitude','longitude']), how='left', on=['latitude','longitude'])
loan_country

,loan_id,latitude,longitude,country
0,657307,8,124,Philippines
1,673156,8,124,Philippines
2,888943,8,124,Philippines
3,613323,8,124,Philippines
4,777937,8,124,Philippines
...,...,...,...,...
1216482,155137,16,121,Philippines
1216483,162583,10,-84,Costa Rica
1216484,172304,9,125,Philippines
1216485,852449,-14,34,Malawi


Ci sono alcune coordinate che non sono state localizzate, in particolare:

In [9]:
missing_ll = loan_country[pd.isnull(loan_country['country'])].drop(columns=['loan_id']).groupby(['latitude','longitude']).size().reset_index()
missing_ll.columns = ['lati sep = ';' sep = ';'tude', 'longitude', 'num']
missing_ll.num.sum()

3327

Guardando su Google Maps, queste sembrerebbero tutte finire nell'oceano. Essendo circa lo 0.003% dei record totali decido di non considerarle.

In [10]:
loan_country = loan_country.dropna().drop(columns=['latitude','longitude'])
loan_country

,loan_id,country
0,657307,Philippines
1,673156,Philippines
2,888943,Philippines
3,613323,Philippines
4,777937,Philippines
...,...,...
1216482,155137,Philippines
1216483,162583,Costa Rica
1216484,172304,Philippines
1216485,852449,Malawi


Controllo che lo status sia "funded"

In [11]:
loans_funded = loans[loans['status'] == 'funded']

In [30]:
country_loans = (pd.merge(loan_country, loans_funded[['loan_id','loan_amount']], how='left', on='loan_id')).drop(columns=['loan_id'])
country_loans = country_loans.groupby('country').size().reset_index()
country_loans.columns = ['country','num_of_loans']
country_loans

,country,num_of_loans
0,Afghanistan,6681
1,Albania,1079
2,Angola,5
3,Argentina,4578
4,Armenia,12927
...,...,...
105,Uruguay,1
106,Vietnam,15828
107,Yemen,3304
108,Zambia,712


### 5. For each country, compute the overall amount of money borrowed.

Controllo che lo status sia "funded"

In [13]:
loans_funded = loans[loans['status'] == 'funded']

Creo un dataframe contenente per ogni paese il totale di soldi finanziati

In [14]:
country_loans = (pd.merge(loan_country, loans_funded[['loan_id','loan_amount']], how='left', on='loan_id')).drop(columns=['loan_id'])
country_loans = country_loans.groupby('country').sum().reset_index()
country_loans.columns = ['country','total_loan']
country_loans

,country,total_loan
0,Afghanistan,5121950.0
1,Albania,1274700.0
2,Angola,3350.0
3,Argentina,11241325.0
4,Armenia,16256600.0
...,...,...
105,Uruguay,8000.0
106,Vietnam,16338950.0
107,Yemen,2573425.0
108,Zambia,1011675.0


### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [15]:
overall = country_loans['total_loan'].sum()
overall

889618925.0

In [16]:
country_loans['total_loan_%'] = country_loans['total_loan']/overall

### 7. Like the three previous points, but split for each year (with respect to disburse time).

In [17]:
loans['disburse_year'] = pd.to_datetime(loans.disburse_time).dt.year
loans_funded = loans[loans['status'] == 'funded']
loans_funded[['loan_id','funded_amount','disburse_year']]

,loan_id,funded_amount,disburse_year
0,657307,125.0,2013.0
1,657259,400.0,2013.0
2,658010,400.0,2014.0
3,659347,625.0,2014.0
4,656933,425.0,2013.0
...,...,...,...
1419602,988180,400.0,2015.0
1419603,988213,300.0,2015.0
1419604,989109,2425.0,2015.0
1419605,989143,100.0,2015.0


In [33]:
country_loans = (pd.merge(loan_country, loans_funded[['loan_id','funded_amount','disburse_year']], how='left', on='loan_id')).drop(columns=['loan_id'])
#country_loans = country_loans.groupby(['country', 'disburse_year']).size().reset_index()
#country_loans.columns = ['country', 'disburse_year', 'num_of_loans']
country_loans

,country,funded_amount,disburse_year
0,Philippines,125.0,2013.0
1,Philippines,675.0,2014.0
2,Philippines,775.0,2015.0
3,Philippines,125.0,2013.0
4,Philippines,NaN,NaN
...,...,...,...
1213155,Philippines,150.0,2009.0
1213156,Costa Rica,1150.0,2009.0
1213157,Philippines,150.0,2010.0
1213158,Malawi,1500.0,2015.0


Non possiedo tutti i disburse_year poichè alcuni disburse_time erano NaN

In [20]:
country_loans = (pd.merge(loan_country, loans_funded[['loan_id','funded_amount','disburse_year']], how='left', on='loan_id')).drop(columns=['loan_id'])
country_loans = country_loans.groupby(['country', 'disburse_year']).sum().reset_index()
country_loans

,country,disburse_year,funded_amount
0,Afghanistan,2007.0,204800.0
1,Afghanistan,2008.0,395900.0
2,Afghanistan,2009.0,698675.0
3,Afghanistan,2010.0,552475.0
4,Afghanistan,2011.0,364250.0
...,...,...,...
889,Zimbabwe,2013.0,307700.0
890,Zimbabwe,2014.0,859575.0
891,Zimbabwe,2015.0,555825.0
892,Zimbabwe,2016.0,680550.0


In [37]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_year
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,2013.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,2013.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,2014.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013.0
